<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/agent/openai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Function Calling Mistral Agent

This notebook shows you how to use our Mistral agent, powered by function calling capabilities.

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.


In [ ]:
%pip install llama-index-llms-mistralai
%pip install llama-index-embeddings-mistralai

In [ ]:
!pip install llama-index

In [12]:
import json
from typing import Sequence, List

from llama_index.llms.mistralai import MistralAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

Let's define some very simple calculator tools for our agent.

In [13]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [14]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [15]:
llm = MistralAI(model="mistral-large-latest")

## Initialize Mistral Agent

Here we initialize a simple Mistral agent with calculator functions.

In [21]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)
agent = AgentRunner(agent_worker)

In [23]:
response = agent.chat("What is (121 * 4) + 3?")
print(str(response))

Added user message to memory: What is (121 * 4) + 3?
MESSAGES: [ChatMessage(role='user', content='What is (121 * 4) + 3?', name=None, tool_calls=None)]
=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 4}
MESSAGES: [ChatMessage(role='user', content='What is (121 * 4) + 3?', name=None, tool_calls=None), ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='null', type=<ToolType.function: 'function'>, function=FunctionCall(name='multiply', arguments='{"a": 121, "b": 4}')), ToolCall(id='null', type=<ToolType.function: 'function'>, function=FunctionCall(name='add', arguments='{"a": 484, "b": 3}'))]), ChatMessage(role='tool', content='484', name=None, tool_calls=None)]



KeyboardInterrupt



### Chat

In [22]:
response = agent.chat("What is (121 * 3) + 5?")
print(str(response))

Added user message to memory: What is (121 * 3) + 5?
MESSAGES: [ChatMessage(role='user', content='What is (121 * 3) + 5?', name=None, tool_calls=None)]
=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 3}
MESSAGES: [ChatMessage(role='user', content='What is (121 * 3) + 5?', name=None, tool_calls=None), ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='null', type=<ToolType.function: 'function'>, function=FunctionCall(name='multiply', arguments='{"a": 121, "b": 3}')), ToolCall(id='null', type=<ToolType.function: 'function'>, function=FunctionCall(name='add', arguments='{"a": 363, "b": 5}'))]), ChatMessage(role='tool', content='363', name=None, tool_calls=None)]


KeyboardInterrupt: 

In [ ]:
# inspect sources
print(response.sources)

[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]


### Async Chat

In [ ]:
response = await agent.achat("What is 121 * 3?")
print(str(response))

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363

STARTING TURN 2
---------------

121 multiplied by 3 is equal to 363.


## Mistral Agent over RAG Pipeline

Build a Mistral agent over a simple 10K document.

In [8]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'

--2024-03-23 11:13:41--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.09s   

2024-03-23 11:13:41 (19.3 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]



In [8]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.llms.mistralai import MistralAI

embed_model = MistralAIEmbedding()
query_llm = MistralAI(model="mistral-medium")

# load data
uber_docs = SimpleDirectoryReader(input_files=["./data/10k/uber_2021.pdf"]).load_data()
# build index
uber_index = VectorStoreIndex.from_documents(uber_docs, embed_model=embed_model)
uber_engine = uber_index.as_query_engine(similarity_top_k=3, llm=query_llm)
query_engine_tool = QueryEngineTool(
    query_engine=uber_engine,
    metadata=ToolMetadata(
        name="uber_10k",
        description=(
            "Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
)

In [9]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools([query_engine_tool], llm=llm, verbose=True)
agent = AgentRunner(agent_worker)

In [11]:
response = agent.chat("Tell me the risk factors for Uber?") 
print(str(response))

Added user message to memory: Tell me the risk factors for Uber?
MESSAGES: [ChatMessage(role='user', content='Tell me the risk factors for Lyft?', name=None, tool_calls=None), ChatMessage(role='assistant', content="I'm sorry for the confusion, but the function provided is related to Uber's financials for the year 2021. I don't have information about Lyft's risk factors. If you have any questions about Uber's financials, I'd be happy to help with that.", name=None, tool_calls=None), ChatMessage(role='user', content='Tell me the risk factors for Uber?', name=None, tool_calls=None)]
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What are the risk factors for Uber?"}
MESSAGES: [ChatMessage(role='system', content="You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid state


KeyboardInterrupt

